In [1]:
import argparse
import csv
import sys

import ijson

from path_to_re.internal.dep_graph import Step, DepGraph
from path_to_re.internal.detokenizer import Detokenizer
from path_to_re.internal.link import Link
from path_to_re.internal.pipe_error_work_around import revert_to_default_behaviour_on_sigpipe
from path_to_re.internal.sync_tac_tags import SyncTacTags
from path_to_re.internal.tupa_parser import TupaParser

In [2]:
model_prefix=r'C:\Users\jyellin\re_1\tupa\models\bert_multilingual_layers_4_layers_pooling_weighted_align_sum'
parser = TupaParser(model_prefix)

Loading from 'C:\Users\jyellin\re_1\tupa\models\bert_multilingual_layers_4_layers_pooling_weighted_align_sum.json'.


[dynet] 2.1-125-g81d70254
INFO:pytorch_pretrained_bert.tokenization:loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-cased-vocab.txt from cache at C:\Users\jyellin\.pytorch_pretrained_bert\96435fa287fbf7e469185f1062386e05a075cadbf6838b74da22bf64b080bc32.99bcd55fc66f4f3360bc49ba472b940b8dcf223ea6a345deb969d607ca900729
INFO:pytorch_pretrained_bert.modeling:loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-cased.tar.gz from cache at C:\Users\jyellin\.pytorch_pretrained_bert\731c19ddf94e294e00ec1ba9a930c69cc2a0fd489b25d3d691373fae4c0986bd.4e367b0d0155d801930846bb6ed98f8a7c23e0ded37888b29caa37009a40c7b9
INFO:pytorch_pretrained_bert.modeling:extracting archive file C:\Users\jyellin\.pytorch_pretrained_bert\731c19ddf94e294e00ec1ba9a930c69cc2a0fd489b25d3d691373fae4c0986bd.4e367b0d0155d801930846bb6ed98f8a7c23e0ded37888b29caa37009a40c7b9 to temp dir C:\Users\jyellin\AppData\Local\Temp\tmpcxc0rarc
INFO

Loading from 'C:\Users\jyellin\re_1\tupa\models\bert_multilingual_layers_4_layers_pooling_weighted_align_sum.enum'... Done (0.058s).
Loading model from 'C:\Users\jyellin\re_1\tupa\models\bert_multilingual_layers_4_layers_pooling_weighted_align_sum': 21param [00:03,  5.52param/s]                   
Loading model from 'C:\Users\jyellin\re_1\tupa\models\bert_multilingual_layers_4_layers_pooling_weighted_align_sum': 100%|████████| 21/21 [00:03<00:00,  5.40param/s]
Loading from 'C:\Users\jyellin\re_1\tupa\models\bert_multilingual_layers_4_layers_pooling_weighted_align_sum.nlp.json'.


In [3]:
output_stream = sys.stdout
csv_writer = csv.writer(output_stream)
csv_writer.writerow(['id', 'docid', 'relation', 'path'])

id,docid,relation,path


In [4]:
input_stream = open(r'C:\Users\jyellin\re_1\tacred\data\json\train.json', encoding='utf-8')
json_stream = ijson.items(input_stream, 'item')

In [5]:
detokenizer = Detokenizer()

In [10]:
import more_itertools
from path_to_re.internal.dep_graph import Step, DepGraph


batch_count = 0

for batch in more_itertools.chunked(filter(lambda item: item['relation'] != 'no_relation', json_stream), 25):

    batch_count += 1
    if batch_count > 1:
        break

    sentences = []
    for item in batch:
        tac_tokens = item['token']
        sentence = detokenizer.detokenize(tac_tokens)
        sentences.append(sentence)

    parsed_sentences = parser.parse_sentences(sentences)

    for item, sentence, parsed_sentence in zip(batch, sentences, parsed_sentences):

        print()

        tac_tokens = item['token']
        ucca_tokens = [ucca_terminal.text for ucca_terminal in parsed_sentence.terminals]

        tac_tokens_lookup = {
            'subj_start': item['subj_start'],
            'subj_end': item['subj_end'],
            'obj_start': item['obj_start'],
            'obj_end': item['obj_end']
        }
        ucca_token_lookup = SyncTacTags.b_lookup_to_a_lookup(ucca_tokens, tac_tokens, tac_tokens_lookup)

        if (len(ucca_token_lookup) != len(tac_tokens_lookup)):
            print('were in trouble')
            continue

        ent1_start = ucca_token_lookup['subj_start'] + 1
        ent1_end = ucca_token_lookup['subj_end'] + 1

        ent2_start = ucca_token_lookup['obj_start'] + 1
        ent2_end = ucca_token_lookup['obj_end'] + 1

        print('sentence: {sentence}'.format(sentence=sentence))
        print('entity1: {entity}'.format(entity=' '.join(ucca_tokens[ent1_start-1:ent1_end])))
        print('entity2: {entity}'.format(entity=' '.join(ucca_tokens[ent2_start-1:ent2_end])))
        
        links = parsed_sentence.get_links()
        
        ent1_start_node_id = parsed_sentence.get_node_id_by_token_id(ent1_start)
        ent1_parent_node_ids = Link.get_parents(links, ent1_start_node_id)
        if len(ent1_parent_node_ids) == 0:
            print('trouble identifying start node for ent1')
            continue
        ent1_parent_node_id = ent1_parent_node_ids[0]

        ent2_start_node_id = parsed_sentence.get_node_id_by_token_id(ent2_start)
        ent2_parent_node_ids = Link.get_parents(links, ent2_start_node_id)
        if len(ent2_parent_node_ids) == 0:
            print('trouble identifying start node for ent2')
            continue
        ent2_parent_node_id = ent2_parent_node_ids[0]

        
        graph = DepGraph(links)
        
        steps = graph.get_undirected_steps(ent1_parent_node_id, ent2_parent_node_id)
        steps_representation = Step.get_default_representation(steps)
        
        print(steps_representation)
        

        wait_here = True




sentence: Eight months after she began the process to adopt little David Banda, the US singer returned to the impoverished southern African nation this week not only with her latest charge but also her biological daughter Lourdes.
entity1: she
entity2: US
<A <H >H >A >E

sentence: "One rule for the rich and famous and different rules for everyone else ," said Najee Ali, director of Project Islamic Hope.
entity1: Project Islamic Hope
entity2: Najee Ali
<E <E <E >C

sentence: Carson , 33, has been a member of the Indianapolis City-County Council since August.
entity1: Carson
entity2: Indianapolis City - County Council
<C <A >A >E >E >E

sentence: Mia Farrow, a UN goodwill ambassador, has labeled the Beijing Games the "genocide Olympics" and has launched her own Olympic - style torch relay through countries with histories of mass atrocities.
entity1: Mia Farrow
entity2: UN
<C >E >E

sentence: It and FirstGroup are the only operators with a nationwide presence across the U.S., finance chi